## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [33]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [34]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [35]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Users\Vincent\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [36]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3, 3), input_shape=x_train.shape[1:], activation = 'relu'))#32,3,3,input_shape=(32,32,3),activation='relu'
classifier.add(BatchNormalization(momentum = 0.99, epsilon = 0.001))
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(32, (3, 3), activation = 'relu'))#32,3,3,input_shape=(32,32,3),activation='relu'
classifier.add(BatchNormalization(momentum = 0.99, epsilon = 0.001))
classifier.add(MaxPooling2D(pool_size=(2,2)))
#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units = 100, activation = 'relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=128,epochs=100)

C:\Users\Vincent\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/100
50000/50000 [==============================] - 42s 845us/step - loss: 1.3944 - accuracy: 0.5100
Epoch 2/100
50000/50000 [==============================] - 39s 786us/step - loss: 1.0005 - accuracy: 0.6479
Epoch 3/100
50000/50000 [==============================] - 41s 826us/step - loss: 0.8356 - accuracy: 0.7080
Epoch 4/100
50000/50000 [==============================] - 45s 905us/step - loss: 0.7198 - accuracy: 0.7483
Epoch 5/100
50000/50000 [==============================] - 44s 870us/step - loss: 0.6286 - accuracy: 0.7816
Epoch 6/100
50000/50000 [==============================] - 43s 866us/step - loss: 0.5500 - accuracy: 0.8079
Epoch 7/100
50000/50000 [==============================] - 40s 799us/step - loss: 0.4784 - accuracy: 0.8349
Epoch 8/100
50000/50000 [==============================] - 42s 838us/step - loss: 0.4205 - accuracy: 0.8544
Epoch 9/100
50000/50000 [==============================] - 46s 915us/step - loss: 0.3615 - accuracy: 0.8752
Epoch 10/100
50000/50000 [==

50000/50000 [==============================] - 40s 793us/step - loss: 0.0326 - accuracy: 0.9879
Epoch 77/100
50000/50000 [==============================] - 40s 793us/step - loss: 0.0323 - accuracy: 0.9889
Epoch 78/100
50000/50000 [==============================] - 40s 792us/step - loss: 0.0262 - accuracy: 0.9911
Epoch 79/100
50000/50000 [==============================] - 40s 792us/step - loss: 0.0304 - accuracy: 0.9899
Epoch 80/100
50000/50000 [==============================] - 40s 798us/step - loss: 0.0268 - accuracy: 0.9907
Epoch 81/100
50000/50000 [==============================] - 40s 792us/step - loss: 0.0270 - accuracy: 0.9909
Epoch 82/100
50000/50000 [==============================] - 40s 794us/step - loss: 0.0267 - accuracy: 0.9904
Epoch 83/100
50000/50000 [==============================] - 40s 792us/step - loss: 0.0284 - accuracy: 0.9904
Epoch 84/100
50000/50000 [==============================] - 40s 792us/step - loss: 0.0287 - accuracy: 0.9903
Epoch 85/100
50000/50000 [======

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [37]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[9.5940224e-05, 3.5423554e-20, 6.3428620e-04, 9.9918884e-01,
        1.2952627e-08, 2.3477873e-14, 1.4602074e-09, 6.0353808e-15,
        8.0903956e-05, 5.4144892e-26]], dtype=float32)